In [ ]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report
from skimage.feature import hog
from skimage.transform import resize


root_path = "C:/Users/Edu/Desktop/CANCER"

def obtener_caracteristicas_hog(img):
    # Resize a 64x64
    img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect', preserve_range=True)

    #Hog de las imagenes redimensionadas antes
    features = hog(img_resized, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), visualize=False)

    return features




print("Cargando imágenes...")
# Leer imagenes/asignar etiquetas
X = []
y = []

#bucle por todas las carpetas
for folder in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder)
    if os.path.isdir(folder_path):
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            if os.path.isdir(subfolder_path):
                for file_name in os.listdir(subfolder_path):
                    if os.path.isfile(os.path.join(subfolder_path, file_name)):
                        img_path = os.path.join(subfolder_path, file_name)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            features = obtener_caracteristicas_hog(img)
                            X.append(features)
                            y.append(int(subfolder))

print("Imágenes cargadas.")

# listas a array numpy
X = np.array(X)
y = np.array(y)

# test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizo los datos
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Creomodelo SVM
clf = svm.SVC(kernel='linear')

print("Entrenando el modelo...")
clf.fit(X_train, y_train)
print("Entrenamiento finalizado.")

print("Haciendo predicciones...")
y_pred = clf.predict(X_test)
print("Predicciones realizadas.")

#Evaluacion
print("Evaluación del modelo:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


